<a href="https://colab.research.google.com/github/NoemiRyf/ML2_Project/blob/main/Trash_Sorting_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary libraries 

In [ ]:
!ls


data		 LICENSE    README.md	 train.lua  weight-init.lua
DataLoader.lua	 model.lua  shuffle.lua  trashnet
dataset-resized  plot.lua   test.lua	 utils.lua


In [41]:
%cd /content

/content


In [43]:
!ls

'archive (7).zip'   dataset.zip   sample_data   trashnet


In [47]:
!unzip archive (7).zip


/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `unzip archive (7).zip'


In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [48]:
!git clone https://github.com/garythung/trashnet.git



Cloning into 'trashnet'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 45 (delta 6), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (45/45), 40.64 MiB | 18.12 MiB/s, done.


In [ ]:
%cd /content/trashnet/data


In [61]:
import zipfile

zip_path = '/content/trashnet/data/dataset-resized.zip'
destination_folder = '/content/trashnet/data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)


In [63]:
!python resize.py


Define image data generator for training and validation data

In [65]:
# Define image data generator for training and validation data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load your dataset
train_set = train_datagen.flow_from_directory('/content/trashnet/data/dataset-resized',
                                              target_size=(64, 64),
                                              batch_size=32,
                                              class_mode='categorical')

Found 2527 images belonging to 6 classes.


Build Model

In [66]:
# Build your model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=6, activation='softmax'))  # 3 units for 3 classes: recyclable, compostable, or trash


Compile Model

In [67]:
# Compile your model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Train Model

In [68]:
# Train your model
model.fit(train_set, epochs=25)

Epoch 1/25
79/79 [==============================] - 14s 170ms/step - loss: 2.0736 - accuracy: 0.3114
Epoch 2/25
79/79 [==============================] - 13s 167ms/step - loss: 1.4238 - accuracy: 0.4278
Epoch 3/25
79/79 [==============================] - 13s 166ms/step - loss: 1.3245 - accuracy: 0.4769
Epoch 4/25
79/79 [==============================] - 13s 168ms/step - loss: 1.2922 - accuracy: 0.5038
Epoch 5/25
79/79 [==============================] - 13s 166ms/step - loss: 1.2201 - accuracy: 0.5299
Epoch 6/25
79/79 [==============================] - 13s 167ms/step - loss: 1.1573 - accuracy: 0.5647
Epoch 7/25
79/79 [==============================] - 14s 178ms/step - loss: 1.1282 - accuracy: 0.5706
Epoch 8/25
79/79 [==============================] - 13s 167ms/step - loss: 1.0778 - accuracy: 0.6035
Epoch 9/25
79/79 [==============================] - 13s 166ms/step - loss: 1.0298 - accuracy: 0.6146
Epoch 10/25
79/79 [==============================] - 13s 167ms/step - loss: 0.9864 - accura

Save Model

In [70]:
# Save your model
model.save('/content/trashnet/trash_classifier.h5')

from google.colab import files

# Download the model file
files.download('/content/trashnet/trash_classifier.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [72]:
from PIL import Image
import numpy as np

# Load and preprocess the test image
test_image_path = '/content/trashnet/test_images/glass.jpg'
test_image = Image.open(test_image_path)
target_size = (64, 64)
test_image = test_image.resize(target_size)
test_image = np.array(test_image) / 255.0
test_image = np.expand_dims(test_image, axis=0)

# Make predictions on the test image
predictions = model.predict(test_image)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Map the predicted class index to the corresponding class label
class_labels = train_set.class_indices
predicted_class_label = {v: k for k, v in class_labels.items()}[predicted_class_index]

# Print the predicted class label
print('Predicted class:', predicted_class_label)


1/1 [==============================] - 0s 85ms/step
Predicted class: glass
